In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

In [4]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an instance of the OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(api_key=api_key, model="gpt-4o")

In [5]:
# Create the coding agent.
coding_agent = AssistantAgent(
    name="CODING_AGENT",
    model_client=model_client,
    system_message="You are a helpful AI assistant expert in the Python Programming. Generate the code for the task.",
)

# Create the testing agent.
testing_agent = AssistantAgent(
    name="TESTING_AGENT",
    model_client=model_client,
    system_message="You are a helpful AI assistant expert in the Python Programming. Test the code for the task." \
        "If the code is good, then reply with 'STOP'",
)                                                                                                           

# Create the coding editor agent.
coding_editor_agent = AssistantAgent(
    name="CODING_EDITOR_AGENT",
    model_client=model_client,
    system_message="You are a helpful AI assistant expert in the Python Programming. Edit the code for the task." \
        "If the code is good, then reply with 'STOP'",
)

In [6]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination


# Termination Condition
termination_condiation = TextMentionTermination('STOP')


# Create the group chat.
team = RoundRobinGroupChat(
    participants=[coding_agent, testing_agent, coding_editor_agent],
    termination_condition=termination_condiation
)

In [7]:
from autogen_agentchat.ui import Console

await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Write a simple Hello world code ?"))  # Stream the messages to the console.

---------- TextMessage (user) ----------
Write a simple Hello world code ?
---------- TextMessage (CODING_AGENT) ----------
Certainly! Here's a simple Python code to print "Hello, World!":

```python
print("Hello, World!")
```

Simply copy and paste this code into a Python environment to run it.
---------- TextMessage (TESTING_AGENT) ----------
STOP


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a simple Hello world code ?', type='TextMessage'), TextMessage(source='CODING_AGENT', models_usage=RequestUsage(prompt_tokens=38, completion_tokens=40), metadata={}, content='Certainly! Here\'s a simple Python code to print "Hello, World!":\n\n```python\nprint("Hello, World!")\n```\n\nSimply copy and paste this code into a Python environment to run it.', type='TextMessage'), TextMessage(source='TESTING_AGENT', models_usage=RequestUsage(prompt_tokens=97, completion_tokens=1), metadata={}, content='STOP', type='TextMessage')], stop_reason="Text 'STOP' mentioned")